In [ ]:
# default_exp rolling

In [ ]:
#hide
%load_ext autoreload
%autoreload 2

# Rolling

> Rolling window operations

In [ ]:
#hide
import random

from nbdev.showdoc import *

In [ ]:
#export
from math import sqrt
from typing import Callable, Optional

import numpy as np
from numba import njit

from window_ops.utils import _gt, _lt, _rolling_std, _validate_rolling_sizes

In [ ]:
import pandas as pd

np.random.seed(0)
y = np.random.rand(100)
ys = pd.Series(y)

## Regular

In [ ]:
#export
@njit
def rolling_mean(x: np.ndarray,
                 window_size: int,
                 min_samples: Optional[int] = None) -> np.ndarray:
    n_samples = x.size
    window_size, min_samples = _validate_rolling_sizes(n_samples, window_size, min_samples)
    out = np.full(n_samples, np.nan, dtype=np.float32)
    accum = 0.
    for i in range(window_size):
        accum += x[i]
        if i + 1 >= min_samples:
            out[i] = accum / (i+1)
    for i in range(window_size, n_samples):
        accum += x[i] - x[i-window_size]
        out[i] = accum / window_size
    return out

In [ ]:
window_size = random.randint(2, 10)
min_samples = random.randint(2, window_size)

assert np.allclose(rolling_mean(y, window_size, min_samples=1), 
                   ys.rolling(window_size, min_periods=1).mean().values)

assert np.allclose(rolling_mean(y, window_size, min_samples=min_samples), 
                   ys.rolling(window_size, min_periods=min_samples).mean().values,
                   equal_nan=True)

In [ ]:
#export
@njit
def rolling_std(x: np.ndarray, 
                window_size: int,
                min_samples: Optional[int] = None) -> np.ndarray:
    out, _, _ = _rolling_std(x, window_size, min_samples)
    return out

In [ ]:
window_size = random.randint(3, 10)
min_samples = random.randint(2, window_size)

assert np.allclose(rolling_std(y, window_size, min_samples=2), 
                   ys.rolling(window_size, min_periods=2).std().values,
                   equal_nan=True)

assert np.allclose(rolling_std(y, window_size, min_samples=min_samples), 
                   ys.rolling(window_size, min_periods=min_samples).std().values,
                   equal_nan=True)

assert np.allclose(rolling_std(y, window_size), 
                   ys.rolling(window_size).std().values,
                   equal_nan=True)

In [ ]:
#exporti
@njit 
def _rolling_comp(comp: Callable,
                  x: np.ndarray, 
                  window_size: int,
                  min_samples: Optional[int] = None): 
    n_samples = x.size   
    window_size, min_samples = _validate_rolling_sizes(n_samples, window_size, min_samples)    
    out = np.full(n_samples, np.nan, dtype=np.float32)  
    for i in range(min_samples - 1, n_samples):
        pivot = x[i]
        ws = min(i+1, window_size)
        for j in range(1, ws):
            if comp(x[i - j], pivot) > 0:
                pivot = x[i - j]
        out[i] = pivot
    return out

In [ ]:
#export
@njit
def rolling_max(x: np.ndarray,
                window_size: int,
                min_samples: Optional[int] = None):
    return _rolling_comp(_gt, x, window_size, min_samples)

In [ ]:
window_size = random.randint(3, 10)
min_samples = random.randint(2, window_size)

assert np.allclose(rolling_max(y, window_size, min_samples=1), 
                   ys.rolling(window_size, min_periods=1).max().values)

assert np.allclose(rolling_max(y, window_size, min_samples=min_samples), 
                   ys.rolling(window_size, min_periods=min_samples).max().values,
                   equal_nan=True)

assert np.allclose(rolling_max(y, window_size),
                   ys.rolling(window_size).max().values,
                   equal_nan=True)

In [ ]:
#export
@njit
def rolling_min(x: np.ndarray,
                window_size: int,
                min_samples: Optional[int] = None):
    return _rolling_comp(_lt, x, window_size, min_samples)

In [ ]:
window_size = random.randint(2, 10)
min_samples = random.randint(2, window_size)

assert np.allclose(rolling_min(y, window_size, min_samples=1), 
                   ys.rolling(window_size, min_periods=1).min().values)

assert np.allclose(rolling_min(y, window_size, min_samples=min_samples), 
                   ys.rolling(window_size, min_periods=min_samples).min().values,
                   equal_nan=True)

assert np.allclose(rolling_min(y, window_size), 
                   ys.rolling(window_size).min().values,
                   equal_nan=True)

## Seasonal

In [ ]:
#exporti
def _seasonal_rolling_op(rolling_op: Callable,
                         x: np.ndarray,
                         season_length: int,
                         window_size: int,
                         min_samples: Optional[int] = None) -> np.ndarray: 
    n_samples = x.size
    out = np.full(n_samples, np.nan, dtype=np.float32)
    for season in range(season_length):
        out[season::season_length] = rolling_op(x[season::season_length], window_size, min_samples)
    return out

In [ ]:
#export
def seasonal_rolling_mean(x: np.ndarray,
                          season_length: int,
                          window_size: int,
                          min_samples: Optional[int] = None) -> np.ndarray:
    return _seasonal_rolling_op(rolling_mean, x, season_length, window_size=window_size, min_samples=min_samples)

In [ ]:
y_df = ys.to_frame('y')
y_df['season'] = np.arange(7)[[i % 7 for i in range(y.size)]]
grouped_y = y_df.groupby('season')['y']

In [ ]:
window_size = random.randint(2, 8)
min_samples = random.randint(1, window_size)

assert np.allclose(grouped_y.transform(lambda y: y.rolling(window_size, min_periods=min_samples).mean()).values,
                   seasonal_rolling_mean(y, 7, window_size, min_samples),
                   equal_nan=True)

In [ ]:
#export
def seasonal_rolling_std(x: np.ndarray,
                         season_length: int,
                         window_size: int,
                         min_samples: Optional[int] = None
                         ) -> np.ndarray:
    return _seasonal_rolling_op(rolling_std, x, season_length, window_size, min_samples)

In [ ]:
window_size = random.randint(4, 8)
min_samples = random.randint(2, window_size)

assert np.allclose(grouped_y.transform(lambda y: y.rolling(window_size, min_periods=min_samples).std()).values,
                   seasonal_rolling_std(y, 7, window_size, min_samples),
                   equal_nan=True)

In [ ]:
#export
def seasonal_rolling_max(x: np.ndarray,
                         season_length: int,
                         window_size: int,
                         min_samples: Optional[int] = None
                         ) -> np.ndarray:
    return _seasonal_rolling_op(rolling_max, x, season_length, window_size, min_samples)

In [ ]:
window_size = random.randint(2, 8)
min_samples = random.randint(1, window_size)

assert np.allclose(grouped_y.transform(lambda y: y.rolling(window_size, min_periods=min_samples).max()).values,
                   seasonal_rolling_max(y, 7, window_size, min_samples),
                   equal_nan=True)

In [ ]:
#export
def seasonal_rolling_min(x: np.ndarray,
                         season_length: int,
                         window_size: int,
                         min_samples: Optional[int] = None
                         ) -> np.ndarray:
    return _seasonal_rolling_op(rolling_min, x, season_length, window_size, min_samples)

In [ ]:
window_size = random.randint(2, 8)
min_samples = random.randint(1, window_size)

assert np.allclose(grouped_y.transform(lambda y: y.rolling(window_size, min_periods=min_samples).min()).values,
                   seasonal_rolling_min(y, 7, window_size, min_samples),
                   equal_nan=True)